In [6]:
import os

from utils.io import load_yaml
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

# load best model
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone/version_0'
configs = load_yaml(os.path.join(prefix, 'configs.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=19.ckpt'), shape=configs['model']['shape'])

In [ ]:
# create dataset and evaluate model on
#   - with motion dataset
#   - robust mis
#   - cholec80
